# Collatz conjecture visualizations

Anton Antonov   
RakuForPrediction blog at WordPress   
May 2025

-----

## Introduction

This notebook presents various visualizations related to the [Collatz conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture) using Raku.

The Raku subs for the Collatz sequences are easy to define. The visualizations are done with the packages
["Graph"](https://raku.land/zef:antononcube/Graph),
["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3), and
["Math::NumberTheory"](https://raku.land/zef:antononcube/Math::NumberTheory).

-----

## Setup

In [ ]:
#%javascript

require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#%js
js-d3-list-line-plot(rand xx 100, background => '#1F1F1F')

In [ ]:
my $background = 'none';
my $stroke-color = 'Ivory';
my $fill-color = 'none';
my $title-color = 'DarkGray';

----

## Definition

In [ ]:
sub collatz(UInt $n is copy, Int:D $max-steps = 1000) {
    return [] if $n == 0;
    my @sequence = $n;
    while $n != 1 && @sequence.elems < $max-steps {
        $n = $n %% 2 ?? $n div 2 !! 3 * $n + 1;
        @sequence.push: $n;
    }
    return @sequence;
}

In [ ]:
collatz(12)

-----

## Simple visualizations

### Collatz sequence numbers

Here is the simplest -- and very informative -- Collatz sequence plot:

In [ ]:
#% js
js-d3-list-line-plot(collatz(27), :$background)

Let us make a multi-lines plot for a selection of integers:

In [ ]:
my @data = (1..1_000).map({ collatz($_) }).grep({ 30 ≤ $_.elems ≤ 150 && $_.max ≤ 600 }).pick(10).sort(*.head).map({my $i = $_.head; $_.kv.map(-> $x, $y {%(group => $i, :$x, :$y )}).Array }).map(*.Slip).Array;

deduce-type(@data)

**Remark:** Using simple sampling like the code block below would generally produce very non-uniform length and max-member sequences.
Hence we do the filtering above.

```raku
my @data = (^100).pick(9).sort.map(-> $i {collatz($i).kv.map(-> $x, $y {%(group => $i, :$x, :$y )}).Array }).map(*.Slip).Array;
```

In [ ]:
#% js
js-d3-list-line-plot(@data.flat, :$background)

-----

## Sequences lengths distribution

Here is a histogram of the Collarz sequences lengths distributions:

In [ ]:
#% js
my $m = 100_000;
my @cLengths = (^$m).map({ collatz($_).elems });
js-d3-histogram(
    @cLengths, 
    100,
    :$background,
    :600width, 
    :400height, 
    title => "Collatz sequences lengths distribution (up to $m)",
    :$title-color
  )

Here is a corresponding 2D plot (i.e. number vs. sequence length):

In [ ]:
#% js
js-d3-list-plot(
    @cLengths, 
    :$background, 
    :2point-size,
    :800width, 
    :400height, 
    title => 'Collatz sequences lengths',
    x-label => 'integer',
    y-label => 'sequence length',
    :$title-color
  )

-------

## Spiral embedding

A certain concentric pattern emerges in the spiral embedding plots of the Collatz sequences lengths `mod 8`.

In [ ]:
#% js
#my @sunflower = sunflower-embedding(12_000, with => { collatz($_).elems mod 8 + 1}):d;
my @sunflower = sunflower-embedding(12_000, with => { collatz($_).elems }):d;

js-d3-list-plot(@sunflower, 
    background => 'none',
    point-size => 4,
    width => 900, height => 440, 
    :!axes, 
    :!legends,
    color-scheme => 'Tableau10',
    margins => {:20top, :20bottom, :250left, :250right}
 )

----

## Graphs

In [ ]:
proto sub collatz-edges(|) {*}

multi sub collatz-edges(Int:D $n) {
    ($n mod 3 == 2) ?? [$n => 2 * $n, $n => (2 * $n - 1) / 3] !! [$n => 2 * $n,]
}

multi sub collatz-edges(@edges where @edges.all ~~ Pair:D) {
    my @leafs = @edges».value.unique;
    @edges.append(@leafs.map({ collatz-edges($_.Int) }).flat)
}

In [ ]:
my @edges = Pair.new(1, 2);

for ^16 { @edges = collatz-edges(@edges) }

deduce-type(@edges)

In [ ]:
my $g = Graph.new(@edges.map({ $_.key.Str => $_.value.Str })):d

In [ ]:
#% html
$g.dot(
    engine => 'twopi',
    :$background,
    vertex-label-color => 'Gray',
    vertex-width => 0.7,
    :24vertex-font-size,
    edge-thickness => 6,
    graph-size => 10
):svg

In [ ]:
#% js
# my %force = y => {strength => 0.5},  charge => {strength => -10, iterations => 2}, collision => {radius => 25, iterations => 4}, link => {distance => 0};

# js-d3-graph-plot(
#     $g.edges,
#     :directed,
#     :$background,
#     vertex-label-color => 'Gray',
#     :1200width, :600height,
#     :%force
# )

----

## References

[AAp1] Anton Antonov,
[Graph Raku package](https://github.com/antononcube/Raku-Graph),
(2024-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[JavaScript::D3 Raku package](https://github.com/antononcube/Raku-JavaScript-D3),
(2022-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[Math::NumberTheory Raku package](https://github.com/antononcube/Raku-Math-NumberTheory),
(2025),
[GitHub/antononcube](https://github.com/antononcube).